### Experiments using IMLY ###

This notebook contains experimental runs of IMLY with different datasets.  
The readings of these experiments can be referred to in this [sheet](https://docs.google.com/spreadsheets/d/1E5jcq2w42gN8bMIaeaRJpAdhgSVN-2XDJ_YTHe4qfwY/edit?usp=sharing).

---

### Dataset  #1

#### Diabetes dataset

In [1]:
import experiment_automation_script
from os import path
import pandas as pd

dataset_info = experiment_automation_script.get_dataset_info("diabetes")
url = "../data/diabetes.csv" if path.exists("../data/diabetes.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, index_col=False)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

experiment_automation_script.dopify(dataset_info, 'linear_regression', X, Y, 0.60)

Using TensorFlow backend.
  0%|          | 0/3 [00:00<?, ?it/s]

From try --  <function glm at 0x000002223E336F28>


 33%|███▎      | 1/3 [00:02<00:04,  2.15s/it]

From try --  <function glm at 0x000002223E336F28>


 67%|██████▋   | 2/3 [00:02<00:01,  1.67s/it]

From try --  <function glm at 0x000002223E336F28>


100%|██████████| 3/3 [00:03<00:00,  1.34s/it]


Scan Finished!
Epoch 1/1
266/266 [==============================] - ETA:  - 0s 116us/step


---

### Dataset  #2

#### UCI Abalone dataset

In [1]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


dataset_info = experiment_automation_script.get_dataset_info("uci_abalone")

names = ["sex", "length", "diameter", "height", "whole weight",
        "shucked weight", "viscera weight", "shell weight", "rings"]
url = "../data/abalone.data.csv" if path.exists("../data/abalone.data.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, names=names, index_col=False)
data.head()

# Check for columns that contain missing values #
col_names = data.columns

num_data = data.shape[0]

categorical_col = ['sex']
for col in categorical_col:
    b, c = np.unique(data[col], return_inverse=True)
    data[col] = c

    
# Filter dataset to contain 'rings' 9 and 10 #
data = data[data['rings'].isin([9,10])]
data['rings'] = data['rings'].map({9: 0, 10: 1})


feature_list = names[:7]
X = data.loc[:, feature_list]
Y = data[['rings']]


experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

Using TensorFlow backend.
C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Keras classifier chosen


C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  0%|          | 0/1 [00:00<?, ?it/s]

From try --  <function glm at 0x0000025C4CBA0048>


100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


Scan Finished!
from elif
From except --  <function glm at 0x0000025C4CBA0048>
Epoch 1/10
529/529 [==============================] - ETA: 2s - loss: 0.7830 - acc: 0.406 - 0s 354us/step - loss: 0.7246 - acc: 0.4726
Epoch 2/10
529/529 [==============================] - ETA: 0s - loss: 0.6524 - acc: 0.562 - 0s 89us/step - loss: 0.7153 - acc: 0.4707
Epoch 3/10
529/529 [==============================] - ETA: 0s - loss: 0.7824 - acc: 0.312 - 0s 89us/step - loss: 0.7082 - acc: 0.4764
Epoch 4/10
529/529 [==============================] - ETA: 0s - loss: 0.7153 - acc: 0.437 - ETA: 0s - loss: 0.7007 - acc: 0.480 - 0s 118us/step - loss: 0.7012 - acc: 0.4783
Epoch 5/10
529/529 [==============================] - ETA: 0s - loss: 0.7122 - acc: 0.468 - 0s 118us/step - loss: 0.6965 - acc: 0.4877
Epoch 6/10
529/529 [==============================] - ETA: 0s - loss: 0.6591 - acc: 0.593 - 0s 89us/step - loss: 0.6924 - acc: 0.4934
Epoch 7/10
529/529 [==============================] - ETA: 0s - loss: 0.7516 

---

### Dataset  #3

#### UCI Iris dataset

In [1]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path

dataset_name = "uci_iris"
dataset_info = experiment_automation_script.get_dataset_info(dataset_name)

url = "../data/iris.csv" if path.exists("../data/iris.csv") else dataset_info['url']
data = pd.read_csv(url , delimiter=",", header=None, index_col=False)
class_name,index = np.unique(data.iloc[:,-1],return_inverse=True)
data.iloc[:,-1] = index
data = data.loc[data[4] != 2]
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

Using TensorFlow backend.


Keras classifier chosen


  0%|          | 0/1 [00:00<?, ?it/s]

From try --  <function glm at 0x0000018AB8D3D048>


100%|██████████| 1/1 [00:02<00:00,  2.70s/it]


Scan Finished!
from elif
From except --  <function glm at 0x0000018AB8D3D048>
Epoch 1/10
40/40 [==============================] - ETA: 0s - loss: 0.4986 - acc: 0.750 - 0s 4ms/step - loss: 0.5196 - acc: 0.7000
Epoch 2/10
40/40 [==============================] - ETA: 0s - loss: 0.5245 - acc: 0.687 - 0s 199us/step - loss: 0.5153 - acc: 0.7000
Epoch 3/10
40/40 [==============================] - ETA: 0s - loss: 0.4964 - acc: 0.750 - 0s 299us/step - loss: 0.5101 - acc: 0.7250
Epoch 4/10
40/40 [==============================] - ETA: 0s - loss: 0.5117 - acc: 0.718 - 0s 274us/step - loss: 0.5057 - acc: 0.7250
Epoch 5/10
40/40 [==============================] - ETA: 0s - loss: 0.5176 - acc: 0.687 - 0s 274us/step - loss: 0.5009 - acc: 0.7250
Epoch 6/10
40/40 [==============================] - ETA: 0s - loss: 0.5151 - acc: 0.687 - 0s 249us/step - loss: 0.4965 - acc: 0.7250
Epoch 7/10
40/40 [==============================] - ETA: 0s - loss: 0.5140 - acc: 0.687 - 0s 274us/step - loss: 0.4922 - acc: 

In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)
np.random.seed(7)

def create_model():
    model = Sequential()
    model.add(Dense(1,input_dim=4,activation='sigmoid'))

    # Compile the model #

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model
    

model = KerasClassifier(build_fn=create_model)
model.fit(x_train, y_train)

scores = model.score(x_test, y_test)
scores

from else
input from __call__  --  Tensor("dense_1_input:0", shape=(?, 4), dtype=float32)
Epoch 1/1
60/60 [==============================] - ETA:  - 0s 931us/step


0.4666666626930237

In [7]:
create_model.__call__()

input from __call__  --  Tensor("dense_2_input_1:0", shape=(?, 4), dtype=float32)


---

### Dataset  #4

#### UCI Adult salary dataset

In [1]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path

dataset_name = "uci_adult_salary"
dataset_info = experiment_automation_script.get_dataset_info(dataset_name)


names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
         'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
         'hours-per-week', 'native-country', 'target']
url = "../data/iris.csv" if path.exists("../data/dataset.csv.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=" ", header=None, names=names)


data = data[data["workclass"] != "?"]
data = data[data["occupation"] != "?"]
data = data[data["native-country"] != "?"]

# Convert categorical fields #
categorical_col = ['workclass', 'education', 'marital-status', 'occupation',
                   'relationship', 'race', 'sex', 'native-country', 'target']

for col in categorical_col:
    b, c = np.unique(data[col], return_inverse=True)
    data[col] = c

feature_list = names[:14]
# Test train split #
X = data.loc[:, feature_list]
Y = data[['target']]

experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

# Split the dataset into test and train datasets

# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

Using TensorFlow backend.
C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Keras classifier chosen


C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  0%|          | 0/1 [00:00<?, ?it/s]

From try --  <function glm at 0x0000022E23B86F28>


100%|██████████| 1/1 [00:31<00:00, 31.83s/it]


Scan Finished!
from elif
From except --  <function glm at 0x0000022E23B86F28>
Epoch 1/10
18088/18088 [==============================] - ETA: 1:36 - loss: 4.5332 - acc: 0.718 - ETA: 6s - loss: 3.6769 - acc: 0.7719  - ETA: 3s - loss: 3.7101 - acc: 0.769 - ETA: 2s - loss: 3.7337 - acc: 0.768 - ETA: 2s - loss: 3.6822 - acc: 0.771 - ETA: 1s - loss: 3.7573 - acc: 0.766 - ETA: 1s - loss: 3.7464 - acc: 0.767 - ETA: 1s - loss: 3.7853 - acc: 0.765 - ETA: 1s - loss: 3.8051 - acc: 0.763 - ETA: 1s - loss: 3.8077 - acc: 0.763 - ETA: 1s - loss: 3.8414 - acc: 0.761 - ETA: 0s - loss: 3.8220 - acc: 0.762 - ETA: 0s - loss: 3.8507 - acc: 0.761 - ETA: 0s - loss: 3.8475 - acc: 0.761 - ETA: 0s - loss: 3.8289 - acc: 0.762 - ETA: 0s - loss: 3.8144 - acc: 0.763 - ETA: 0s - loss: 3.8425 - acc: 0.761 - ETA: 0s - loss: 3.8558 - acc: 0.760 - ETA: 0s - loss: 3.8683 - acc: 0.760 - ETA: 0s - loss: 3.8731 - acc: 0.759 - ETA: 0s - loss: 3.8738 - acc: 0.759 - ETA: 0s - loss: 3.8766 - acc: 0.759 - ETA: 0s - loss: 3.8831 -

18088/18088 [==============================] - ETA: 0s - loss: 2.5185 - acc: 0.843 - ETA: 1s - loss: 3.7777 - acc: 0.765 - ETA: 1s - loss: 3.7002 - acc: 0.770 - ETA: 1s - loss: 3.8389 - acc: 0.761 - ETA: 1s - loss: 3.8927 - acc: 0.758 - ETA: 1s - loss: 3.9841 - acc: 0.752 - ETA: 1s - loss: 3.9616 - acc: 0.754 - ETA: 1s - loss: 3.9240 - acc: 0.756 - ETA: 1s - loss: 3.9205 - acc: 0.756 - ETA: 1s - loss: 3.9288 - acc: 0.756 - ETA: 1s - loss: 3.9274 - acc: 0.756 - ETA: 0s - loss: 3.9181 - acc: 0.756 - ETA: 0s - loss: 3.9002 - acc: 0.758 - ETA: 0s - loss: 3.9099 - acc: 0.757 - ETA: 0s - loss: 3.9508 - acc: 0.754 - ETA: 0s - loss: 3.9193 - acc: 0.756 - ETA: 0s - loss: 3.9122 - acc: 0.757 - ETA: 0s - loss: 3.9372 - acc: 0.755 - ETA: 0s - loss: 3.9109 - acc: 0.757 - ETA: 0s - loss: 3.9299 - acc: 0.756 - ETA: 0s - loss: 3.9103 - acc: 0.757 - ETA: 0s - loss: 3.9177 - acc: 0.756 - ETA: 0s - loss: 3.8972 - acc: 0.758 - ETA: 0s - loss: 3.8891 - acc: 0.758 - ETA: 0s - loss: 3.8724 - acc: 0.759 - ETA

---

### Dataset  #5

#### UCI Ad dataset

In [1]:
import experiment_automation_script
import pandas as pd
import numpy as np
from os import path
from sklearn.preprocessing import LabelEncoder

dataset_name = "uci_ad"
dataset_info = experiment_automation_script.get_dataset_info(dataset_name)

url = "../data/ad.data.csv" if path.exists("../data/dataset.csv.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, index_col=False)

# Check for columns that contain missing values #

data = data.applymap(lambda val: np.nan if str(val).strip() == '?' else val)
data = data.dropna()


# Label encoding #

lb = LabelEncoder()
Y = lb.fit_transform(data.iloc[:, -1])

X = data.iloc[:,:-1]

# Normalize the X values #
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

experiment_automation_script.dopify(dataset_info, 'logistic_regression', X, Y, 0.60)

Using TensorFlow backend.
C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Keras classifier chosen


C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  0%|          | 0/1 [00:00<?, ?it/s]

From try --  <function glm at 0x000001C8E1236F28>


100%|██████████| 1/1 [00:03<00:00,  3.71s/it]


Scan Finished!
from elif
From except --  <function glm at 0x000001C8E1236F28>
Epoch 1/10
943/943 [==============================] - ETA: 4s - loss: 0.7525 - acc: 0.500 - ETA: 0s - loss: 0.7753 - acc: 0.589 - ETA: 0s - loss: 0.6939 - acc: 0.658 - 0s 274us/step - loss: 0.6936 - acc: 0.6564
Epoch 2/10
943/943 [==============================] - ETA: 0s - loss: 0.4893 - acc: 0.781 - ETA: 0s - loss: 0.4560 - acc: 0.846 - 0s 135us/step - loss: 0.3951 - acc: 0.8791
Epoch 3/10
943/943 [==============================] - ETA: 0s - loss: 0.3706 - acc: 0.906 - ETA: 0s - loss: 0.2988 - acc: 0.921 - 0s 116us/step - loss: 0.2698 - acc: 0.9374
Epoch 4/10
943/943 [==============================] - ETA: 0s - loss: 0.1768 - acc: 0.968 - ETA: 0s - loss: 0.2124 - acc: 0.951 - 0s 116us/step - loss: 0.2017 - acc: 0.9512
Epoch 5/10
943/943 [==============================] - ETA: 0s - loss: 0.1759 - acc: 0.937 - ETA: 0s - loss: 0.1599 - acc: 0.968 - 0s 116us/step - loss: 0.1606 - acc: 0.9629
Epoch 6/10
943/943 

---

### Dataset  #6

#### UCI Mushroom dataset

In [3]:
from sklearn.model_selection import train_test_split
import copy

model_name = 'linear_regression'
model_mappings = {
    'linear_regression': 'LinearRegression',
    'logistic_regression': 'LogisticRegression'
}

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

for key, value in model_mappings.items():
    if key == model_name:
        name = value

module = __import__('sklearn.linear_model', fromlist=[name])
imported_module = getattr(module, name)
model = imported_module

primal_model = model()

# Primal
primal_model.fit(x_train, y_train)
primal_model.__class__.__name__

'LinearRegression'

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

diabetes = datasets.load_diabetes()

# Use only one feature
diabetes_X = diabetes.data
# sc = StandardScaler()
# diabetes.data = sc.fit_transform(diabetes.data)

X = diabetes.data
Y = diabetes.target

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

# # Split the data into training/testing sets
# x_train = diabetes_X[:-20]
# x_test = diabetes_X[-20:]

# # Split the targets into training/testing sets
# y_train = diabetes.target[:-20]
# y_test = diabetes.target[-20:]


In [23]:
# temp_data = np.column_stack([X,Y])
# np.savetxt("diabetes.csv", temp_data, delimiter=",")

In [3]:
import winmltools
winmltools.__file__

'C:\\Users\\shakk\\Anaconda2\\envs\\py36\\lib\\site-packages\\winmltools\\__init__.py'

In [9]:
import onnxmltools

def f1(**kwargs):
    params_json = json.load(open('../imly/architectures/sklearn/params.json'))
    params = params_json['params']
    kwargs.setdefault('params', params)
    kwargs.setdefault('x_train', np.array([[1], [2]]))

    model = Sequential()
    model.add(Dense(kwargs['params']['first_neuron'],
                    input_dim=kwargs['x_train'].shape[1],
                    activation=kwargs['params']['activation']))

    model.compile(optimizer=kwargs['params']['optimizer'],
                  loss=kwargs['params']['losses'],
                  metrics=['acc'])
    onnx_model = onnxmltools.convert_keras(model, target_opset=8)
    print(type(model))
    onnx_model
    return onnx_model

In [10]:
import json
import numpy as np
from keras import Sequential
from keras.layers import Dense

model = f1()

The maximum opset needed by this model is only 7.


<class 'keras.engine.sequential.Sequential'>


In [19]:
type(model)
# cross check import (f1p1 and f2p2 combination) - Is it possible to edit after the export-import flow

onnx.onnx_ml_pb2.ModelProto

In [18]:
import onnx
onnx.save(model, './onnx_model.onnx')

---

## Test bed ##

In [40]:
mapping = { "KerasClassifier": {
    "LogisticRegression": True
},
 "KerasRegressor": {
     "LinearRegression": True
 }
}

name = "LinearRegression"

for key, value in mapping.items():
    test = mapping[key]
    print(test)

{'LogisticRegression': True}
{'LinearRegression': True}


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense
import numpy as np
import json


def glm(**kwargs):  # Should param_name be optional or mandatory?

    # kwargs.setdefault('param_name', 'glm_1')
    params_json = json.load(open('../imly/architectures/sklearn/params.json')) # Remove and make it generic
    params = params_json['params'][kwargs['param_name']]
    kwargs.setdefault('params', params)
    kwargs.setdefault('x_train', np.array([[1], [2]]))

    model = Sequential()
    model.add(Dense(kwargs['params']['first_neuron'], # Change first_neuron to input_size
                    input_dim=kwargs['x_train'].shape[1], # Find a better way to pass input_dim. Through params maybe?
                    activation=kwargs['params']['activation']))

    model.compile(optimizer=kwargs['params']['optimizer'],
                  loss=kwargs['params']['losses'],
                  metrics=["accuracy"])
    return model

In [6]:
glm.__call__(param_name="log_reg")